## Подбор параметров модели методом поиска на сетке - GridSearch 

Поиск оптимальных значений ключевых параметров модели (то есть значений, которые дают наилучшую обобщающую
способность) является сложной задачей, но она обязательна почти для всех моделей и наборов данных. Поскольку поиск оптимальных значений параметров является общераспространенной задачей, библиотека **scikit-learn** предлагает стандартные методы, позволяющие решить ее.

Наиболее часто используемый метод – это решетчатый поиск (**grid search**), который по сути является перебором всех возможных комбинаций интересующих параметров.


Рассмотрим модель дерева решений DecisionTreeClassifier на известном наборе данных и оценим качество классификации при различных значениях параметров модели:

* min_samples_leaf - минимальное кол-во примеров в листе (по умолчанию 1)
* min_samples_split - минимальное кол-во примеров для ветвления (по умолчанию 2)
* max_depth - максимальная глубина дерева (по умоланию не ограничено)

Значения по умолчанию часто приводят к заметному перебочунию дерева решений (на обучающей выборке 1.0, а на тестовой значительно ниже)

Покрутим эти параметры по сетке из следующих значений:

* min_samples_leaf = {1, 2, ..10}
* min_samples_split = {2,4, ... 10}
* max_depth = {2,3,...20}


Класс **GridSearchCV** выполняет подбор параметров указанной модели на заданной сетке значений с кросс-валидацией модели, поскольку хорошие результаты для набора параметров могли быть получены на случайно удачном наборе данных, потому для проверки помимо разбиения на train/test для каждой модели необходимо также выделить валидационную часть общего набора данных, либо выполнять кросс-валидацию, которую **GridSearchCV** и выполняет.

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()

In [2]:
dtc = DecisionTreeClassifier()

params = { 'max_depth': range (2, 20, 1),
              'min_samples_leaf': range (1, 10),
              'min_samples_split': range (2,10,2) }

grid = GridSearchCV(dtc, params, cv = 5, n_jobs = -1)
grid.fit(iris.data, iris.target)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(2, 20),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10, 2)})

In [3]:
print(f"Наилучшие значения параметров: {grid.best_params_}")
print(f"Наилучшее значение метрики: {grid.best_score_}")

Наилучшие значения параметров: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 4}
Наилучшее значение метрики: 0.9733333333333334


<hr>

## Ансамбли моделей. Случайный лес (RandomForestClassifier)

В машинном обучении под ансамблем моделей понимают комбинацию нескольких алгоритмов обучения, которые, работая вместе, позволяют построить модель более эффективную и точную, чем любая из моделей, построенная с помощью отдельного алгоритма.

случайный лес – это набор деревьев решений, где каждое дерево немного отличается от остальных. Идея случайного леса заключается в том, что каждое дерево может довольно хорошо прогнозировать, но скорее всего переобучается на части данных. Если мы построим много деревьев, которые хорошо работают и переобучаются с разной степенью, мы можем уменьшить переобучение путем усреднения их результатов.

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, train_size = 0.75)

In [5]:
forest = RandomForestClassifier(n_estimators = 5)
forest.fit(X_train, y_train)

RandomForestClassifier(n_estimators=5)

In [6]:
print("Точность на обучающем наборе: {:.3f}".format(forest.score(X_train, y_train)))
print("Точность на тестовом наборе: {:.3f}".format(forest.score(X_test, y_test)))

Точность на обучающем наборе: 0.991
Точность на тестовом наборе: 0.974


In [7]:
params = {  "n_estimators": range(3,20),
            'max_depth': range (2, 20, 1),
            'min_samples_leaf': range (1, 10),
            'min_samples_split': range (2,10,2) }

gridRf = GridSearchCV(forest, params, cv = 5, n_jobs = -1)
gridRf.fit(iris.data, iris.target)

GridSearchCV(cv=5, estimator=RandomForestClassifier(n_estimators=5), n_jobs=-1,
             param_grid={'max_depth': range(2, 20),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10, 2),
                         'n_estimators': range(3, 20)})

In [8]:
print(f"Наилучшие значения параметров: {gridRf.best_params_}")
print(f"Наилучшее значение метрики: {gridRf.best_score_}")

Наилучшие значения параметров: {'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 3}
Наилучшее значение метрики: 0.9733333333333334


## Ансамбли моделей. Бэггинг (Bagging) и Бустинг (Boosting)


Помимо случайного леса существует несколько типов ансамблей моделей, но два наиболее распространенных - это бэггинг и бустинг.

Бэггинг (Bagging) - это метод, который основан на создании нескольких независимых моделей на основе разных подмножеств обучающих данных. Каждая модель обучается на своем подмножестве данных и в результате выдает свой прогноз. Затем, прогнозы всех моделей усредняются, чтобы получить окончательный результат. Использование бэггинга позволяет уменьшить влияние шумовых данных и увеличить стабильность и точность предсказаний.

Бустинг (Boosting) - это метод, который похож на бэггинг, но с одним основным отличием: модели создаются последовательно, и каждая следующая модель использует информацию об ошибках предыдущей модели для улучшения своих прогнозов. Это позволяет уменьшить ошибки, которые делает каждая модель, и увеличить точность предсказаний.

Кроме того, существуют и другие типы ансамблей моделей, такие как случайный лес (Random Forest), который является вариантом бэггинга, но использует решающие деревья в качестве базовых моделей, и градиентный бустинг (Gradient Boosting), который является одной из разновидностей бустинга, но использует градиентный спуск для минимизации ошибок.

В целом, использование ансамблей моделей может значительно улучшить качество предсказаний в машинном обучении, особенно если выбранные модели являются достаточно разнообразными и хорошо настроены.

### BaggingClassifier

**BaggingClassifier** - это классификатор на основе бэггинга, доступный в библиотеке **scikit-learn**. Он позволяет создавать ансамбли моделей на основе разных алгоритмов машинного обучения с использованием бэггинга.


Основные параметры: 
* base_estimator - это базовый алгоритм, на основе которого создается ансамбль (в данном случае -  К ближайших соседей), 
* n_estimators - количество моделей в ансамбле, 
* random_state - начальное значение для генератора случайных чисел, чтобы обеспечить воспроизводимость результатов.

**BaggingClassifier** используется с различными алгоритмами машинного обучения в качестве базовых моделей. Например, вместо решающего дерева можно использовать логистическую регрессию или метод ближайших соседей. 

Важно помнить, что выбор базового алгоритма должен основываться на особенностях конкретной задачи и на том, какой тип модели лучше всего подходит для решения этой задачи.

Рассмотрим пример использования классификатора на основе алгоритма К ближайших соседей:


In [3]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# создаем синтетические данные для примера
X, y = make_classification( n_samples = 1000, n_features = 10 )

# разделяем данные на обучающие и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2 )

# создаем базовый алгоритм KNeighborsClassifier
base_model = KNeighborsClassifier()

# создаем ансамбль на основе базового алгоритма с использованием бэггинга
bagging_model = BaggingClassifier( base_estimator = base_model, n_estimators = 10, random_state = 42 )

# обучаем модель на обучающих данных
bagging_model.fit( X_train, y_train )

# делаем предсказания на тестовых данных
y_pred = bagging_model.predict( X_test )




### Градиентный бустинг. BoostingClassifier

**BoostingClassifier** - это алгоритм ансамбля машинного обучения, который комбинирует несколько слабых моделей, чтобы получить более сильную модель. Он работает путем последовательного обучения моделей на разных подмножествах данных, с тем чтобы каждая последующая модель сконцентрировалась на тех объектах, на которых предыдущие модели ошибались. В итоге получается ансамбль, который показывает более высокую точность предсказаний, чем отдельные модели.

Рассмотрим пример, аналогичный Бэггингу с испольщзованием алгоритма AdaBoosting для дерева решений:


In [6]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# создаем синтетические данные для примера
X, y = make_classification( n_samples = 1000, n_features = 10 )

# разделяем данные на обучающие и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2 )

# создаем базовый алгоритм DecisionTreeClassifier
base_model = DecisionTreeClassifier()

# создаем ансамбль на основе базового алгоритма с использованием boosting
boosting_model = AdaBoostClassifier( base_estimator = base_model, n_estimators = 10, random_state = 42 )

# обучаем модель на обучающих данных
boosting_model.fit( X_train, y_train )

# делаем предсказания на тестовых данных
y_pred = boosting_model.predict( X_test )


помимо **AdaBoostClassifier** можно воспользоваться **GradientBoostingClassifier**. Это два разных алгоритма градиентного бустинга, которые используются для построения ансамблей моделей машинного обучения.

Основное отличие между ними заключается в том, как они обновляют веса объектов в процессе построения ансамбля.

**AdaBoostClassifier** обновляет веса объектов на каждой итерации, увеличивая веса объектов, на которых были сделаны ошибки, и уменьшая веса объектов, на которых были сделаны правильные предсказания. Новый базовый алгоритм строится на основе этих обновленных весов. В итоге, объекты, на которых было сделано больше ошибок, получают более высокий вес, и следующая модель в ансамбле более фокусируется на этих объектах.

**GradientBoostingClassifier** также обновляет веса объектов, но делает это путем минимизации функции потерь, используя градиентный спуск. В отличие от ***AdaBoostClassifier**, **GradientBoostingClassifier** не просто корректирует веса объектов, но также учитывает градиенты функции потерь, чтобы найти наилучшее направление обновления весов. Базовые алгоритмы в **GradientBoostingClassifier** строятся на основе этих новых весов, и ансамбль строится путем последовательного добавления базовых алгоритмов с учетом их вклада в функцию потерь.

Таким образом, **AdaBoostClassifier** и **GradientBoostingClassifier** оба используют градиентный спуск, чтобы строить ансамбли моделей, но делают это с разной целью и способом обновления весов объектов. 
* **AdaBoostClassifier** сосредотачивается на тех объектах, на которых было сделано больше ошибок, в то время как 
* **GradientBoostingClassifier** учитывает градиенты функции потерь, чтобы наилучшим образом обновить веса объектов и улучшить общее качество модели.